In [1]:
import os
import sys

# Configure the environment
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\spark-1.4.1-bin-hadoop2.4\spark-1.4.1-bin-hadoop2.4'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

# Add the PySpark/py4j to the Python Path
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib/py4j-0.8.2.1-src.zip"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))
execfile(os.path.join(SPARK_HOME, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.4.1
      /_/

Using Python version 2.7.10 (default, May 28 2015 16:44:52)
SparkContext available as sc, HiveContext available as sqlContext.


In [3]:
# Custom Paritioner 1
data = sc.parallelize(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],2).map(lambda x: (x,x))
print data.collect()
c = data.count()
wp = data.partitionBy(c,lambda k: int(k))
print wp.map(lambda t: t[0]).glom().collect()
# Custom Paritioner 2
data = sc.parallelize(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],2).map(lambda x: (x,x))
print data.collect()
wp = data.partitionBy(2,lambda k: int(k)%2)
print wp.map(lambda t: t[0]).glom().collect()

[('1', '1'), ('2', '2'), ('3', '3'), ('4', '4'), ('5', '5'), ('6', '6'), ('7', '7'), ('8', '8'), ('9', '9'), ('0', '0')]
[['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']]
[('1', '1'), ('2', '2'), ('3', '3'), ('4', '4'), ('5', '5'), ('6', '6'), ('7', '7'), ('8', '8'), ('9', '9'), ('0', '0')]
[['2', '4', '6', '8', '0'], ['1', '3', '5', '7', '9']]


In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
conf = SparkConf() 
(conf.setMaster("local").setAppName("Partitions").set("spark.driver.memory", "512M"))
sc = SparkContext(conf=conf)

In [9]:
# Return the number of Paritions
rdd = sc.parallelize([(1, 2,3, 4)],2)
rdd.getNumPartitions()

2

In [10]:
# Default partition is one for local data
rdd = sc.parallelize([(1, 2,3, 4)])
rdd.getNumPartitions()

1

In [15]:
# Return a new RDD by applying a function to each partition of this RDD.
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator): 
    yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

In [16]:
# Return a new RDD by applying a function to each partition of this RDD, while tracking the index of the original partition.
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator):
    yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()

6

In [17]:
# Return a new RDD by applying a function to each partition of this RDD, while tracking the index of the original partition.
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): 
    yield splitIndex
rdd.mapPartitionsWithSplit(f).sum()

6